In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import inspect
import ast
import numpy as np
import typing

In [25]:
import typist
from typist import type_check

In [26]:

# Todo:
# Add a decorator for an explicit type check. 
@type_check
def example_function(a: int, b: int, c: float, ext = None):
    d = a + b
    e = a + c
    
    return d

source_code = inspect.getsource(example_function)
tree = ast.parse(source_code)
annotated_tree = typist.AnnotateVariableTypes().visit(tree)
annotated_source_code = ast.unparse(annotated_tree)
print(annotated_source_code)

In [28]:
example_function(1,5,3)

TypeError: Argument 'c' should be of type <class 'float'>, but got <class 'int'>

In [16]:
function_def = tree.body[0]
arg1 = function_def.args.args[0]

In [18]:
arg1.annotation.id

'int'

In [9]:
print(ast.dump(annotated_tree, indent=2))

Module(
  body=[
    FunctionDef(
      name='example_function',
      args=arguments(
        posonlyargs=[],
        args=[],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        AnnAssign(
          target=Name(id='a', ctx=Store()),
          annotation=Name(id='int', ctx=Load()),
          value=Constant(value=1),
          simple=1),
        AnnAssign(
          target=Name(id='b', ctx=Store()),
          annotation=Name(id='float', ctx=Load()),
          value=Constant(value=2.0),
          simple=1),
        AnnAssign(
          target=Name(id='c', ctx=Store()),
          annotation=Name(id='str', ctx=Load()),
          value=Constant(value='hello'),
          simple=1),
        Assign(
          targets=[
            Name(id='d', ctx=Store())],
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='b', ctx=Load())))],
      decorator_list=[])],
  type_ignores=[])
